In [48]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import time

In [49]:
batch_size=100

In [64]:
transforms=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))])
                                                   

#Load MNIST dataset from PyTorch torchvision dataset
train_dataset = torchvision.datasets.MNIST(root='data/', 
                                           train=True,
                                           transform=transforms,
                                           download=True)
test_dataset = torchvision.datasets.MNIST(root='data/', 
                                           train=False,
                                          transform=transforms)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                          batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size,
                                          shuffle=False)

In [65]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1=nn.Linear(input_size, hidden_size)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(hidden_size, num_classes)
        #self.softmax = nn.Softmax(dim=1) #calculates softmax across the columns
        
    def forward(self, x):
        out=self.fc1(x)
        out=self.relu(out)
        out=self.fc2(out)
        #out=self.softmax(out)
        return out

In [66]:
input_size=784
hidden_size=500
num_classes=10
num_epochs=5
learning_rate=0.001
scheduler_step_size=10
scheduler_gamma=0.1

In [67]:
#CPU Version
model = NeuralNet(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss() # CrossEntropyLoss which includes Softmax
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step_size,
                                           gamma=scheduler_gamma)
total_step = len(train_loader)
start_time = time.time()

for epoch in range(num_epochs):
    scheduler.step()
    correct=0
    total=0
    for images, labels in train_loader:
        #Move tensors to the configured device
        images=images.reshape(-1, 28*28)
        labels=labels
        #Forward Pass
        outputs=model(images)
        loss=criterion(outputs, labels)
        """The higher the energy for a class, 
        the more the network thinks that the image is of the particular class. 
        So, let’s get the index of the highest energy"""
        _,predicted=torch.max(outputs.data,1) #Returns the maximum value of all elements in the input tensor
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()
        #Backward and optimize
        optimizer.zero_grad() # zero the gradient buffers
        loss.backward() #backpropagate the error
        optimizer.step() # Does the update
    train_accuracy=correct/total
    '''You can also stop autograd from tracking history on Tensors with 
    .requires_grad=True by wrapping the code block in with torch.no_grad():'''
    with torch.no_grad(): #will make all the operations in the block have no gradients
        correct=0
        total=0
        for images, labels in test_loader:
            images=images.reshape(-1, 28*28)
            labels=labels
            outputs=model(images)
            _,predicted=torch.max(outputs.data,1)
            total+=labels.size(0)
            correct+=(predicted==labels).sum().item()
    test_accuracy=correct/total
    
    print('Epoch {}, Time {:.4f}, Loss: {:.4f}, Train Accuracy: {:.4f}, Test Accuracy: {:.4f}'
          .format(epoch, time.time()-start_time, loss.item(), train_accuracy, test_accuracy))
#torch.save(model.state_dict(), 'epoch-{}.ckpt'.format(epoch))

Epoch 0, Time 18.8033, Loss: 1.2936, Train Accuracy: 0.5998, Test Accuracy: 0.7942
Epoch 1, Time 37.7337, Loss: 0.8469, Train Accuracy: 0.8190, Test Accuracy: 0.8482
Epoch 2, Time 59.1815, Loss: 0.6329, Train Accuracy: 0.8536, Test Accuracy: 0.8709
Epoch 3, Time 81.1598, Loss: 0.6121, Train Accuracy: 0.8677, Test Accuracy: 0.8819
Epoch 4, Time 102.5196, Loss: 0.5625, Train Accuracy: 0.8777, Test Accuracy: 0.8877
